![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps%2FServing&file=Understanding+Prediction+IO+With+FastAPI.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/Understanding%20Prediction%20IO%20With%20FastAPI.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FServing%2FUnderstanding%2520Prediction%2520IO%2520With%2520FastAPI.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Serving/Understanding%20Prediction%20IO%20With%20FastAPI.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Serving/Understanding%20Prediction%20IO%20With%20FastAPI.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Understanding Prediction IO With FastAPI

**IN PROGRESS**

Use a simple FastAPI implementation to explore the processing of different input instance format on:
- Vertex AI Endpoints
- Cloud Run Endpoints
- Vetex AI Batch Prediction

Inputs to try:
- [Vertex AI **Online Prediction** Instance Formats](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions#formatting-prediction-input)
- [Vertex AI **Batch Prediction** Instance Formats](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#input_data_requirements)


---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [1]:
# tuples of (import name, install name, min_version)
packages = [
    ('docker', 'docker'),
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.artifactregistry_v1', 'google-cloud-artifact-registry'),
    ('google.cloud.devtools', 'google-cloud-build'),
    ('google.cloud.run_v2', 'google-cloud-run'),   
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [2]:
!gcloud services enable artifactregistry.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable run.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [3]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [4]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [5]:
REGION = 'us-central1'
SERIES = 'mlops-serving'
EXPERIMENT = 'understand-io'

GCS_BUCKET = PROJECT_ID

packages:

In [72]:
import json, os, base64
import time
import requests

import docker

import google.auth
from google.cloud import storage
from google.cloud import artifactregistry_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import run_v2
from google.cloud import aiplatform

clients:

In [7]:
# gcs storage client
gcs = storage.Client(project = GCS_BUCKET)
bucket = gcs.bucket(GCS_BUCKET)

# cloud build client
cb = cloudbuild_v1.CloudBuildClient()

# artifact registry client
ar = artifactregistry_v1.ArtifactRegistryClient()

# cloud run client
cr = run_v2.ServicesClient()

# vertex ai client
aiplatform.init(project = PROJECT_ID, location = REGION)

Parameters:

In [8]:
DIR = f"files/{EXPERIMENT}"

Environment:

In [9]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Build A Custom Prediction Container

It is really not all that hard with Python!

For this example [FastAPI](https://fastapi.tiangolo.com/) is used.

This process uses docker to build a custom container and then runs the container locally, on Cloud Run, and Vertex AI Endpoints.

This could be done locally with Docker and pushed to Artifact Registry before deployment to Cloud Run.  The process below assumes that docker is not available locally and uses Cloud Build to both build and push the resulting container to Artifact Registry.

---
### Setup Artifact Registry

[Artifact registry](https://cloud.google.com/artifact-registry/docs) organizes artifacts with repositories.  Each repository contains packages and is designated to hold a partifcular format of package: Docker images, Python Packages and [others](https://cloud.google.com/artifact-registry/docs/supported-formats#package).

#### List Repositories

This may be empty if no repositories have been created for this project

In [10]:
for repo in ar.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

projects/statmike-mlops-349915/locations/us-central1/repositories/frameworks
projects/statmike-mlops-349915/locations/us-central1/repositories/frameworks-catboost
projects/statmike-mlops-349915/locations/us-central1/repositories/gcf-artifacts
projects/statmike-mlops-349915/locations/us-central1/repositories/mlops
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python


#### Create/Retrieve Docker Image Repository

Create an Artifact Registry Repository to hold Docker Images created by this notebook.  First, check to see if it is already created by a previous run and retrieve it if it has.  Otherwise, create one named for this project.

In [11]:
docker_repo = None
for repo in ar.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    if f'{SERIES}' == repo.name.split('/')[-1]:
        docker_repo = repo
        print(f'Retrieved existing repo: {docker_repo.name}')

if not docker_repo:
    operation = ar.create_repository(
        request = artifactregistry_v1.CreateRepositoryRequest(
            parent = f'projects/{PROJECT_ID}/locations/{REGION}',
            repository_id = f'{SERIES}',
            repository = artifactregistry_v1.Repository(
                description = f'A repository for the {SERIES} series that holds docker images.',
                name = f'{SERIES}',
                format_ = artifactregistry_v1.Repository.Format.DOCKER,
                labels = {'series': SERIES}
            )
        )
    )
    print('Creating Repository ...')
    docker_repo = operation.result()
    print(f'Completed creating repo: {docker_repo.name}')

Creating Repository ...
Completed creating repo: projects/statmike-mlops-349915/locations/us-central1/repositories/mlops-serving


In [12]:
docker_repo.name, docker_repo.format_.name

('projects/statmike-mlops-349915/locations/us-central1/repositories/mlops-serving',
 'DOCKER')

In [13]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

In [14]:
REPOSITORY

'us-central1-docker.pkg.dev/statmike-mlops-349915/mlops-serving'

---
### Create Application Files

```
|__ Dockerfile
|__ requirements.txt
|__ app
    |__ __init__.py
    |__ main.py
    |__ prestart.sh
```

In [15]:
if not os.path.exists(DIR + '/source/app'):
    os.makedirs(DIR + '/source/app')

In [16]:
%%writefile {DIR}/source/Dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY ./requirements.txt requirements.txt

RUN pip install --no-cache-dir --upgrade pip \
  && pip install --no-cache-dir -r requirements.txt

Writing files/understand-io/source/Dockerfile


In [17]:
%%writefile {DIR}/source/requirements.txt
numpy

Writing files/understand-io/source/requirements.txt


In [18]:
%%writefile {DIR}/source/app/__init__.py
# init file

Writing files/understand-io/source/app/__init__.py


In [19]:
%%writefile {DIR}/source/app/main.py
# a simple passthrough of instance to predictions

# packages
import os
from fastapi import FastAPI, Request
import numpy as np

# clients
app = FastAPI()

# Define function for health route
@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}

# Define function for prediction route
@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    # await the request
    body = await request.json()
    
    # parse the request
    instances = body["instances"]
    
    # return the received inputs as the "predictions" - a simple pass through
    predictions = instances

    # this returns just the predicted probabilities:
    return {"predictions": predictions}

Writing files/understand-io/source/app/main.py


In [20]:
%%writefile {DIR}/source/app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing files/understand-io/source/app/prestart.sh


In [21]:
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/Dockerfile').upload_from_filename(f'{DIR}/source/Dockerfile')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/requirements.txt').upload_from_filename(f'{DIR}/source/requirements.txt')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/__init__.py').upload_from_filename(f'{DIR}/source/app/__init__.py')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/main.py').upload_from_filename(f'{DIR}/source/app/main.py')
bucket.blob(f'{SERIES}/{EXPERIMENT}/source/app/prestart.sh').upload_from_filename(f'{DIR}/source/app/prestart.sh')

In [22]:
list(bucket.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}/source'))

[<Blob: statmike-mlops-349915, mlops-serving/understand-io/source/Dockerfile, 1740159897721281>,
 <Blob: statmike-mlops-349915, mlops-serving/understand-io/source/app/__init__.py, 1740159897918644>,
 <Blob: statmike-mlops-349915, mlops-serving/understand-io/source/app/main.py, 1740159897981494>,
 <Blob: statmike-mlops-349915, mlops-serving/understand-io/source/app/prestart.sh, 1740159898044525>,
 <Blob: statmike-mlops-349915, mlops-serving/understand-io/source/requirements.txt, 1740159897860198>]

---
### Build Application Container

Use the Cloud Build client to construct and run the build instructions. Here the files collected in GCS are copied to the build instance, then the Docker build is run in the folder with the `Dockerfile`. The resulting image is pushed to Artifact Registry (setup above).

In [23]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# retrieve the source
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{GCS_BUCKET}/{SERIES}/{EXPERIMENT}/source/*', '/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}', '/workspace']
    }    
)
# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}"]

In [24]:
operation = cb.create_build(
    project_id = PROJECT_ID,
    build = build
)

In [25]:
build_response = operation.result()
build_response.status, build_response.artifacts

(<Status.SUCCESS: 3>,
 images: "us-central1-docker.pkg.dev/statmike-mlops-349915/mlops-serving/understand-io")

In [26]:
build_response.artifacts.images[0]

'us-central1-docker.pkg.dev/statmike-mlops-349915/mlops-serving/understand-io'

---
## Test Locally

If Docker is installed and running locally then use it to test the image.

In [27]:
try:
    local_test = True
    docker_client = docker.from_env()
    if docker_client.ping():
        print(f"Docker is installed and running. Version: {docker_client.version()['Version']}")
except Exception as e:
    local_test = False
    print('Docker is either not installed or not running - please fix before proceeding.\nhttps://docs.docker.com/engine/install/')

Docker is installed and running. Version: 20.10.17


### Pull and Run Container

Run the container image with:
- ports: inside 8080 mapped to outside 80
- set environment variables for:
    - `AIP_HTTP_PORT` is `8080`
    - `AIP_HEALTH_ROUTE` is `/health`
    - `AIP_PREDICT_ROUTE` is `/predict`
    - `AIP_STORAGE_URI` is the `gs://bucket/path/to/folder`
    - `MODULE_NAME` is 'main'
        - this actually defaults to main so is not required
        - an alternative script with different prediction output is created in `main2.py` above
        - use this environment variable to start the container using the alternative script in module `main2`
        - see the [FastAPI Docker Image Advanced Usage](https://github.com/tiangolo/uvicorn-gunicorn-fastapi-docker?tab=readme-ov-file#advanced-usage) details

In [61]:
if local_test:
    # make sure any prior runs are stopped:
    try:
        container = docker_client.containers.get('local-run')
        container.stop()
        container.remove()
    except docker.errors.NotFound:
        pass
    
    # get image:
    image_uri = build_response.artifacts.images[0]
    try:
        local_image = docker_client.images.get(image_uri)
        remote_image = docker_client.images.pull(image_uri)
        if local_image.id != remote_image.id:
            print('New image found, updating ...')
            local_image = remote_image
        else:
            print('Using existing image ...')
    except docker.errors.ImageNotFound:
        print('Pulling image ...')
        local_image = docker_client.images.pull(image_uri)
        
    # run container:
    print('Starting container ...')
    container = docker_client.containers.run(
        image = image_uri,
        detach = True,
        ports = {'8080/tcp':80}, # Map inside:outside (where docker run -p is outside:inside)
        name = 'local-run',
        environment = {
            'AIP_HTTP_PORT': '8080',
            'AIP_HEALTH_ROUTE': '/health',
            'AIP_PREDICT_ROUTE': '/predict',
            'AIP_STORAGE_URI': f'gs://{bucket.name}/{SERIES}/catboost-overview',
            'MODULE_NAME': 'main' # try main2 for alternative output
        }
    )
    print('Container ready.\n\tUse `container.logs()` to view startup logs.')

Using existing image ...
Starting container ...
Container ready.
	Use `container.logs()` to view startup logs.


In [62]:
#container.logs()

### Health Check

Want to see `200`:

In [63]:
if local_test:
    response = requests.get(f"http://localhost:80/health")
    print(response.status_code)

200


### Inference Test

In [64]:
def predict(instances):
    url = f"http://localhost:80/predict"
    headers = {'Content_Type': 'application/json'}
    data = json.dumps({'instances': instances})
    response = requests.post(url, headers = headers, data = data)    
    return json.loads(response.text)

In [65]:
predict([1])

{'predictions': [1]}

In [66]:
predict([1,2,3,4])

{'predictions': [1, 2, 3, 4]}

In [67]:
predict([[1, 2], [3, 4]])

{'predictions': [[1, 2], [3, 4]]}

In [71]:
predict([[[1, 2], [3, 4]]])

{'predictions': [[[1, 2], [3, 4]]]}

In [68]:
predict([{'key':'value'}])

{'predictions': [{'key': 'value'}]}

In [69]:
predict([{'key':'value'}, {'key':'value2'}])

{'predictions': [{'key': 'value'}, {'key': 'value2'}]}

In [70]:
predict(['gs://bucket/path/to/image/image1.jpg'])

{'predictions': ['gs://bucket/path/to/image/image1.jpg']}

In [77]:
b64_string = base64.b64encode(b'something to encode as binary here').decode('utf-8')
b64_string

'c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ=='

In [78]:
predict([b64_string])

{'predictions': ['c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ==']}

In [79]:
predict([{'b64': b64_string}])

{'predictions': [{'b64': 'c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ=='}]}

### Stop Container

In [80]:
container.name

'local-run'

In [81]:
container = docker_client.containers.get(container.name)

In [82]:
container.status

'running'

In [83]:
container.stop()
container.remove()

---
## Vertex AI Prediction Endpoint

Register the model in the [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction) and [Deploy it to an endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment).

### Model Registry

Check for existing version of the model:

In [84]:
parent_model = ''
for model in aiplatform.Model.list(filter=f'display_name="{SERIES}"'):
    parent_model = model.resource_name
    break
parent_model

''

### Upload Model To Registry As New Version

In [85]:
vertex_model = aiplatform.Model.upload(
    display_name = SERIES,
    model_id = SERIES,
    parent_model = parent_model,
    serving_container_image_uri = build_response.artifacts.images[0],
    artifact_uri = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}',
    is_default_version = True,
    version_aliases = [f'{EXPERIMENT}'],
    version_description = EXPERIMENT,
    labels = {'series': SERIES, 'experiment': EXPERIMENT}
)

Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/mlops-serving/operations/967218491801206784
Model created. Resource name: projects/1026793852137/locations/us-central1/models/mlops-serving@1
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/mlops-serving@1')


### Create Endpoint

Check for existing endpoint:

In [86]:
vertex_endpoint = None
for endpoint in aiplatform.Endpoint.list(filter=f'display_name="{SERIES}"'):
    vertex_endpoint = endpoint
    break
vertex_endpoint

Create endpoint if missing:

In [87]:
if not vertex_endpoint:
    vertex_endpoint = aiplatform.Endpoint.create(
        display_name = SERIES,
        labels = {'series': SERIES}   
    )
vertex_endpoint

Creating Endpoint
Create Endpoint backing LRO: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384/operations/664914366813962240
Endpoint created. Resource name: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1026793852137/locations/us-central1/endpoints/1262139293128720384')


resource name: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384

### Deploy Model: Default version with `main.py` - Plain

Get the latest model version with alias `plain`:

In [88]:
vertex_model = aiplatform.Model(model_name = SERIES, version = f'{EXPERIMENT}')
vertex_model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/mlops-serving@1'

In [89]:
vertex_model.deploy(
    endpoint = vertex_endpoint,
    traffic_percentage = 100,
    machine_type = 'n1-standard-4',
    min_replica_count = 1,
    max_replica_count = 2,
)

Deploying model to Endpoint : projects/1026793852137/locations/us-central1/endpoints/1262139293128720384
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384/operations/6491446384724541440
Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384


resource name: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384

### Inference Test

In [93]:
vertex_endpoint.predict([1])

Prediction(predictions=[1.0], deployed_model_id='4861192694560456704', metadata=None, model_version_id='1', model_resource_name='projects/1026793852137/locations/us-central1/models/mlops-serving', explanations=None)

In [ ]:
vertex_endpoint.predict([1]).predictions

In [94]:
vertex_endpoint.predict([1,2,3,4]).predictions

[1.0, 2.0, 3.0, 4.0]

In [95]:
vertex_endpoint.predict([[1, 2], [3, 4]]).predictions

[[1.0, 2.0], [3.0, 4.0]]

In [96]:
vertex_endpoint.predict([[[1, 2], [3, 4]]]).predictions

[[[1.0, 2.0], [3.0, 4.0]]]

In [97]:
vertex_endpoint.predict([{'key':'value'}]).predictions

[{'key': 'value'}]

In [98]:
vertex_endpoint.predict([{'key':'value'}, {'key':'value2'}]).predictions

[{'key': 'value'}, {'key': 'value2'}]

In [99]:
vertex_endpoint.predict(['gs://bucket/path/to/image/image1.jpg']).predictions

['gs://bucket/path/to/image/image1.jpg']

In [100]:
b64_string = base64.b64encode(b'something to encode as binary here').decode('utf-8')
b64_string

'c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ=='

In [101]:
vertex_endpoint.predict([b64_string]).predictions

['c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ==']

In [102]:
vertex_endpoint.predict([{'b64': b64_string}]).predictions

[{'b64': 'c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ=='}]

### Undeploy All Models

In [103]:
for deployed_model in vertex_endpoint.list_models():
    vertex_endpoint.undeploy(deployed_model_id = deployed_model.id)
vertex_endpoint.list_models()

Undeploying Endpoint model: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384
Undeploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384/operations/1409134205236936704
Endpoint model undeployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384


[]

### Delete Endpoint

In [104]:
vertex_endpoint.delete(force = True)

Deleting Endpoint : projects/1026793852137/locations/us-central1/endpoints/1262139293128720384
Endpoint deleted. . Resource name: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384
Deleting Endpoint resource: projects/1026793852137/locations/us-central1/endpoints/1262139293128720384
Delete Endpoint backing LRO: projects/1026793852137/locations/us-central1/operations/7289709418675961856
Endpoint resource projects/1026793852137/locations/us-central1/endpoints/1262139293128720384 deleted.


---
## Cloud Run

Deploy the model to [Cloud Run](https://cloud.google.com/run/docs/overview/what-is-cloud-run) using the same container build and tested above from Artifact Registry.

Some highlights for Cloud Run:
- Rapid scaling to handle requests
- Scale to zero (default) or other minimum if set
- Can handle larger input (request) and output (response) sizes
    - See [requests limits](https://cloud.google.com/run/quotas#request_limits)
- Configure [memory limits](https://cloud.google.com/run/docs/configuring/services/memory-limits) and [cpu limits](https://cloud.google.com/run/docs/configuring/services/cpu) and [concurrency](https://cloud.google.com/run/docs/configuring/concurrency) and [autoscaling](https://cloud.google.com/run/docs/about-instance-autoscaling) and [request timeout](https://cloud.google.com/run/docs/configuring/request-timeout)

### Deploy The Endpoint

In [105]:
parent = f"projects/{PROJECT_ID}/locations/{REGION}"
service = run_v2.Service()
#service.name = f"{parent}/services/{SERIES}"
service.template.containers = [
    run_v2.Container(
        image = build_response.artifacts.images[0],
        ports = [run_v2.ContainerPort(container_port = 8080)],
        env = [
            run_v2.EnvVar(name = 'AIP_HTTP_PORT', value = '8080'),
            run_v2.EnvVar(name = 'AIP_HEALTH_ROUTE', value = '/health'),
            run_v2.EnvVar(name = 'AIP_PREDICT_ROUTE', value = '/predict'),
            run_v2.EnvVar(name = 'AIP_STORAGE_URI', value = f'gs://{bucket.name}/{SERIES}/{EXPERIMENT}'),
            run_v2.EnvVar(name = 'MODULE_NAME', value = 'main')
        ],
        resources = run_v2.ResourceRequirements(
            limits = {"cpu": '8', "memory": '32Gi'}
        )
    )
]
service.ingress = run_v2.IngressTraffic.INGRESS_TRAFFIC_INTERNAL_ONLY

In [106]:
try:
    # create the service:
    run_response = cr.create_service(request = {"parent": parent, "service": service, "service_id": SERIES})
    # wait on the operation to complete:
    run_response.result()
    # print the name of the service
    print(f"Started Service: {run_response.metadata.name}")
except Exception as e:
    print(f"Error creating service: {e}")

Started Service: projects/statmike-mlops-349915/locations/us-central1/services/mlops-serving


In [107]:
run_response.metadata.uri

'https://mlops-serving-urlxi72dpa-uc.a.run.app'

### Permissions

The endpoint requires authentication.  Check ou tthe [Authentication Overview](https://cloud.google.com/run/docs/authenticating/overview) and in the case below the [Authenticating service-to-service](https://cloud.google.com/run/docs/authenticating/service-to-service) method is used by giving the same service account used to run the notebook and create the endpoint the role to invoke the endpoint as well.

In [108]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

In [109]:
run_response.metadata.name

'projects/statmike-mlops-349915/locations/us-central1/services/mlops-serving'

In [110]:
policy = cr.get_iam_policy(request = {'resource': run_response.metadata.name})
policy.bindings.add(
    role = 'roles/run.invoker',
    members = [f"serviceAccount:{SERVICE_ACCOUNT}", 'allUsers'] #'allUsers'
)
policy_response = cr.set_iam_policy(request = {"resource": run_response.metadata.name, "policy": policy})
print(f"IAM policy updated: {policy_response.bindings}")

IAM policy updated: [role: "roles/run.invoker"
members: "allUsers"
members: "serviceAccount:1026793852137-compute@developer.gserviceaccount.com"
]


In [111]:
policy.bindings

[role: "roles/run.invoker"
members: "serviceAccount:1026793852137-compute@developer.gserviceaccount.com"
members: "allUsers"
]

**WAIT: The update of the IAM Policy might take a few moments to take effect.  Rerun the following health check section until you get a `200` response code.**

### Health Check

Want to see `200`:

In [112]:
def health(uri):
    url = f"{uri}/health"
    credentials, _ = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    credentials.refresh(auth_req)
    headers = {'Authorization': f'Bearer {credentials.token}'}
    response = requests.get(url, headers = headers)    
    return response.status_code

def check_health(uri, timeout_seconds = 200, retry_seconds = 10):
    start_time = time.time()
    while True:
        status_code = health(uri)
        if status_code == 200:
            break
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout_seconds:
            break
        time.sleep(retry_seconds)
    return status_code

In [113]:
check_health(run_response.metadata.uri)

200

In [114]:
health(run_response.metadata.uri)

200

### Inference Test

In [115]:
def predict(instances):
    credentials, _ = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    credentials.refresh(auth_req)
    url = f"{run_response.metadata.uri}/predict"
    headers = {
        'Authorization': f'Bearer {credentials.token}',
        'Content_Type': 'application/json'
    }
    data = json.dumps({'instances': instances})
    response = requests.post(url, headers = headers, data = data)    
    return json.loads(response.text)

In [116]:
predict([1])

{'predictions': [1]}

In [117]:
predict([1,2,3,4])

{'predictions': [1, 2, 3, 4]}

In [118]:
predict([[1, 2], [3, 4]])

{'predictions': [[1, 2], [3, 4]]}

In [119]:
predict([[[1, 2], [3, 4]]])

{'predictions': [[[1, 2], [3, 4]]]}

In [120]:
predict([{'key':'value'}])

{'predictions': [{'key': 'value'}]}

In [121]:
predict([{'key':'value'}, {'key':'value2'}])

{'predictions': [{'key': 'value'}, {'key': 'value2'}]}

In [122]:
predict(['gs://bucket/path/to/image/image1.jpg'])

{'predictions': ['gs://bucket/path/to/image/image1.jpg']}

In [123]:
b64_string = base64.b64encode(b'something to encode as binary here').decode('utf-8')
b64_string

'c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ=='

In [124]:
predict([b64_string])

{'predictions': ['c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ==']}

In [125]:
predict([{'b64': b64_string}])

{'predictions': [{'b64': 'c29tZXRoaW5nIHRvIGVuY29kZSBhcyBiaW5hcnkgaGVyZQ=='}]}